# Shifted Kerr-Schild Solution Initial Data

**Authors: George Vopal and Zachariah Etienne**

## This module sets up Shifted Kerr-Schild initial data ([Etienne et al., 2017 GiRaFFE](https://arxiv.org/pdf/1704.00599.pdf)).

### NRPy+ Source Code for this module: [BSSN/BrillLindquist.py](../edit/BSSN/BrillLindquist.py)

<font color='green'>**All quantities have been validated against the [original SENR code](https://bitbucket.org/zach_etienne/nrpy).**</font>

<a id='top'></a>$$\label{top}$$

### Introduction

Kerr-Schild coordinates are similar to the trumpet spacetime. The radial shift both reduces the black hole's coordinate size and causes the very strongly-curved spacetime fields at $r<r_{0}$ to vanish deep inside the horizon, which aids in numerical stability when evolving hydrodynamic, MHD, and FFE fields inside the horizon.


### Table of Contents:  

1. Preliminaries
    1. [Step 1](#step1): Import the core NRPy+ modules that we need, set the dimensionality of the grid with parameter $\text{grid::DIM}$, and declare some of the basic gridfunctions
1. The Kerr-Schild Lapse, Shift, and 3-Metric
    1. [Step 2.1](#step2p1): Define $\rho^{2}$, $\alpha$, $\beta^{r}$, $\beta^{\theta}$, $\beta^{\phi}$, $\gamma_{r\theta}$, $\gamma_{\theta\phi}$
    1. [Step 2.2](#step2p2): Define and construct nonzero components of $\gamma_{ij}$
1. Extrinsic curvature $K_{ij}$
    1. [Step 3.1](#step3p1): Define useful quantities $A$, $B$, $C$
    1. [Step 3.2](#step3p2): Define and construct nonzero components of $K_{ij}$
1. Code Validation
    1. [Step 4](#step4): NRPy+ Module Code Validation

# Preliminaries

First, we will import the core modules of NRPy that we will need and specify the main gridfunctions we will need. 
<a id='step1'></a>

## Step 1: Set up the needed NRPy+ infrastructure and declare core gridfunctions \[Back to [top](#top)\]

**Input for initial data**:

* The black hole mass $M$.
* The black hole spin parameter $a$
$$\label{step1}$$


In [1]:
import sympy as sp
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
from outputC import *
import reference_metric as rfm

# All gridfunctions will be written in terms of spherical coordinates (r, th, ph):
r,th,ph = sp.symbols('r th ph', real=True)

thismodule = "ShiftedKerrSchild"

DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Input parameters:
M = par.Cparameters("REAL", thismodule, ["M"])
a = par.Cparameters("REAL", thismodule, ["a"])

# Auxiliary variables:
rho2 = sp.symbols('rho2', real=True)

<a id='step2p1'></a>

## Step 2.1: Define $\rho^{2}$, $\alpha$, $\beta^{r}$, $\beta^{\theta}$, $\beta^{\phi}$, $\gamma_{r\theta}$, $\gamma_{\theta\phi}$ \[Back to [top](#top)\]
$$\label{step2p1}$$

All of these equations are as defined in the appendix of the original GiRaFFE paper ([Etienne et al., 2017 GiRaFFE](https://arxiv.org/pdf/1704.00599.pdf)).
<br>
First, we define $\rho^{2}$ as

<br>

$$ \rho^2 = r^2 + a^{2}\cos^{2}(\theta) $$

<br>

And we then define the Kerr-Schild lapse $\alpha$ from equation (A.1)

<br>

$$ \alpha = \frac{1}{\sqrt{1 + \frac{2Mr}{\rho^2}}} $$

<br>

And the shift $\beta$ from equations (A.2) & (A.3)

<br>

$$ \beta^{r} = \alpha^2\frac{2Mr}{\rho^2} $$

<br>

$$ \beta^{\theta} = \beta^{\phi} = \gamma_{r\theta} = \gamma_{\theta\phi} = 0 $$

In [2]:
# rho^2 = r^2 + a^2*cos^2(theta)
rho2 = r*r + a*a*sp.cos(th)**2

# alpha = 1/sqrt{1 + Mr/rho^2}
alphaSph = 1/(sp.sqrt(1 + 2*M*r/rho2))

betaSphU = ixp.zerorank1()
# beta^r = alpha^2*2Mr/rho^2
betaSphU[0] = alphaSph*alphaSph*2*M*r/rho2
           
# beta^{theta} = beta^{phi} = gamma_{r theta} = gamma_{theta phi}
BSphU = ixp.zerorank1()
              
gammaSphDD = ixp.zerorank2()

<a id='step2p2'></a>

## Step 2.2: Define and construct nonzero components $\gamma_{rr}$, $\gamma_{r\phi}$, $\gamma_{\theta\theta}$, $\gamma_{\phi\phi}$ \[Back to [top](#top)\]
$$\label{step2p2}$$

From equations (A.4)-(A.7) we define the nonzero components of the 3-metric:

<br>

$$ \gamma_{rr} = 1 + \frac{2Mr}{\rho^2} $$

<br>

$$ \gamma_{r\phi} = -a\gamma_{rr}\sin^2(\theta) $$

<br>

$$ \gamma_{\theta\theta} = \rho^2 $$

<br>

$$ \gamma_{\phi\phi} = \left(r^2 + a^2 + \frac{2Mr}{\rho^2}a^{2}\sin^{2}(\theta)\right)\sin^{2}(\theta) $$

In [3]:
# gammaDD{r r} = 1 +2Mr/rho^2
gammaSphDD[0][0] = 1 + 2*M*r/rho2

# gammaDD{r phi} = -a*gammaDD{r r}*sin^2(theta)
gammaSphDD[0][2] = -a*gammaSphDD[0][0]*sp.sin(th)**2

# gammaDD{theta theta} = rho^2
gammaSphDD[1][1] = rho2

# gammaDD{phi phi} = (r^2 + a^2 + 2Mr/rho^2*a^2*sin^2(theta))*sin^2(theta)
gammaSphDD[2][2] = (r*r + a*a + 2*M*r*a*a*sp.sin(th)**2/rho2)*sp.sin(th)**2

<a id='step3p1'></a>

## Step 3.1: Define useful quantities $A$, $B$, $C$ \[Back to [top](#top)\]

$$\label{step3p1}$$

From equations (A.8)-(A.10) we define the following expressions which will help simplify the nonzero extrinsic curvature components:

<br>

$$ A = \left(a^{2}\cos(2\theta) + a^{2} + 2r^{2}\right) $$

<br>

$$ B = A + 4Mr $$

<br>

$$ D = \sqrt{\frac{2Mr}{a^{2}\cos^{2}(\theta) + r^2} + 1} $$


In [4]:
# *** Define Useful Quantities A, B, D ***
# A = (a^2*cos^2(2theta) + a^2 + 2r^2)
A = (a*a*sp.cos(2*th) + a*a + 2*r*r)

# B = A + 4Mr
B = A + 4*M*r

# D = \sqrt(2Mr/(a^2cos^2(theta) + r^2) + 1)
D = sp.sqrt(2*M*r/(a*a*sp.cos(th)**2 + r*r) + 1)

<a id='step3p2'></a>

## Step 3.2: Define and construct nonzero components of $K_{ij}$ \[Back to [top](#top)\]

$$\label{step3p2}$$
We will now express the extrinsic curvature $K_{ij}$ in spherical polar coordinates.

From equations (A.11) - (A.13) we define the following:

$$ K_{rr} = \frac{D(A + 2Mr)}{A^{2}B}\left[4M\left(a^{2}\cos(2\theta) + a^{2} - 2r^{2}\right)\right] $$

<br>

$$ K_{r\theta} = \frac{D}{AB}\left[8a^{2}Mr\sin(\theta)\cos(\theta)\right] $$

<br>

$$ K_{r\phi} = \frac{D}{A^2}\left[-2aM\sin^{2}(\theta)\left(a^{2}\cos(2\theta) + a^{2} - 2r^{2}\right)\right] $$

In [5]:
# *** The extrinsic curvature in spherical polar coordinates ***

# Establish the 3x3 zero-matrix
KSphDD = ixp.zerorank2()

# *** Fill in the nonzero components ***
# *** This will create an upper-triangular matrix ***
# K_{r r} = D(A+2Mr)/(A^2*B)[4M(a^2*cos(2theta) + a^2 - 2r^2)]
KSphDD[0][0] = D*(A+2*M*r)/(A*A*B)*(4*M*(a*a*sp.cos(2*th)+a*a-2*r*r))

# K_{r theta} = D/(AB)[8a^2*Mr*sin(theta)cos(theta)]
KSphDD[0][1] = D/(A*B)*(8*a*a*M*r*sp.sin(th)*sp.cos(th))

# K_{r phi} = D/A^2[-2aMsin^2(theta)(a^2cos(2theta)+a^2-2r^2)]
KSphDD[0][2] = D/(A*A)*(-2*a*M*sp.sin(th)**2*(a*a*sp.cos(2*th)+a*a-2*r*r))

And from equations (A.14) - (A.17) we define the following expressions to complete the upper-triangular matrix $K_{ij}$:

$$ K_{\theta\theta} = \frac{D}{B}\left[4Mr^{2}\right] $$

<br>

$$ K_{\theta\phi} = \frac{D}{AB}\left[-8a^{3}Mr\sin^{3}(\theta)\cos(\theta)\right] $$

<br>

$$ K_{\phi\phi} = \frac{D}{A^{2}B}\left[2Mr\sin^{2}(\theta)\left(a^{4}(r-M)\cos(4\theta) + a^{4}(M + 3r) + 4a^{2}r^{2}(2r - M) + 4a^{2}r\cos(2\theta)\left(a^{2} + r(M + 2r)\right) + 8r^{5}\right)\right] $$

In [6]:
# K_{theta theta} = D/B[4Mr^2]
KSphDD[1][1] = D/B*(4*M*r*r)

# K_{theta phi} = D/(AB)*(-8*a^3*Mr*sin^3(theta)cos(theta))
KSphDD[1][2] = D/(A*B)*(-8*a**3*M*r*sp.sin(th)**3*sp.cos(th))

# K_{phi phi} = D/(A^2*B)[2Mr*sin^2(theta)(a^4(M+3r)
#   +4a^2r^2(2r-M)+4a^2r*cos(2theta)(a^2+r(M+2r))+8r^5)]
KSphDD[2][2] = D/(A*A*B)*(2*M*r*sp.sin(th)**2*(a**4*(r-M)*sp.cos(4*th)\
                        + a**4*(M+3*r)+4*a*a*r*r*(2*r-M)\
                        + 4*a*a*r*sp.cos(2*th)*(a*a + r*(M + 2*r)) + 8*r**5))

<a id='step4'></a>

## Step 4: NRPy+ Module Code Validation \[Back to [top](#top)\]
$$\label{step4}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for shifted Kerr-Schild initial data between

1. this tutorial and 
2. the NRPy+ [BSSN.ShiftedKerrSdhild](../edit/BSSN/ShiftedKerrSchild.py) module.

In [7]:
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []

# First we import needed core NRPy+ modules
import reference_metric as rfm
import grid as gri
# Unused, but needed because BrillLindquist() also does tensor rescalings.
rfm.reference_metric()

import BSSN.ShiftedKerrSchild as sks
sks.ShiftedKerrSchild()

# It is SAFE to ignore the warning(s) from re-initializing parameters.
print("^^^ Ignore the minor warning(s) above. ^^^\n")
print("Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module:")
print("alphaSph - sks.alphaSph: Should be zero: "+str(sp.simplify(alphaSph - sks.alphaSph)))
for i in range(DIM):
    print("betaSphU["+str(i)+"] - sks.betaSphU["+str(i)+"]: Should be zero: "+\
          str(sp.simplify(betaSphU[i] - sks.betaSphU[i])))
    print("BSphU["+str(i)+"] - sks.BaSphU["+str(i)+"]: Should be zero: "+str(sp.simplify(BSphU[i] - sks.BSphU[i])))
    for j in range(DIM):
        print("gammaSphDD["+str(i)+"]["+str(j)+"] - sks.gammaSphDD["+str(i)+"]["+str(j)+"]: Should be zero: "+\
             str(sp.simplify(gammaSphDD[i][j] - sks.gammaSphDD[i][j])))
        print("KSphDD["+str(i)+"]["+str(j)+"] - sks.KSphDD["+str(i)+"]["+str(j)+"]: Should be zero: "+\
              str(sp.simplify(KSphDD[i][j] - sks.KSphDD[i][j])))

initialize_param() minor warning: Did nothing; already initialized parameter ShiftedKerrSchild::M
initialize_param() minor warning: Did nothing; already initialized parameter ShiftedKerrSchild::a
^^^ Ignore the minor warning(s) above. ^^^

Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module:
alphaSph - sks.alphaSph: Should be zero: 0
betaSphU[0] - sks.betaSphU[0]: Should be zero: 0
BSphU[0] - sks.BaSphU[0]: Should be zero: 0
gammaSphDD[0][0] - sks.gammaSphDD[0][0]: Should be zero: 0
KSphDD[0][0] - sks.KSphDD[0][0]: Should be zero: 0
gammaSphDD[0][1] - sks.gammaSphDD[0][1]: Should be zero: 0
KSphDD[0][1] - sks.KSphDD[0][1]: Should be zero: 0
gammaSphDD[0][2] - sks.gammaSphDD[0][2]: Should be zero: 0
KSphDD[0][2] - sks.KSphDD[0][2]: Should be zero: 0
betaSphU[1] - sks.betaSphU[1]: Should be zero: 0
BSphU[1] - sks.BaSphU[1]: Should be zero: 0
gammaSphDD[1][0] - sks.gammaSphDD[1][0]: Should be zero: 0
KSphDD[1][0] - sks.KSphDD[1][0]: Should be zero: 0
ga

### Output to $\LaTeX$, then PDF

In [8]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-ADM_Initial_Data-ShiftedKerrSchild.ipynb
!pdflatex -interaction=batchmode Tutorial-ADM_Initial_Data-ShiftedKerrSchild.tex
!pdflatex -interaction=batchmode Tutorial-ADM_Initial_Data-ShiftedKerrSchild.tex
!pdflatex -interaction=batchmode Tutorial-ADM_Initial_Data-ShiftedKerrSchild.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-ADM_Initial_Data-ShiftedKerrSchild.ipynb to latex
[NbConvertApp] Writing 43355 bytes to Tutorial-ADM_Initial_Data-ShiftedKerrSchild.tex


This is pdfTeX, Version 3.14159265-2.6-1.40.20 (MiKTeX 2.9.6960 64-bit)
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (MiKTeX 2.9.6960 64-bit)
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (MiKTeX 2.9.6960 64-bit)
entering extended mode


'rm' is not recognized as an internal or external command,
operable program or batch file.
